In [1]:
# import pandas as pd
# df = pd.read_parquet("expanded_df.parquet")
# df['当前法律状态'].value_counts().to_csv('法律状态.csv', encoding='utf-8-sig')
# df = df[df['当前法律状态'] == '专利申请权、专利权的转移']
# df.to_parquet("专利申请权、专利权的转移.parquet")

In [2]:
import pandas as pd
df = pd.read_parquet("专利申请权、专利权的转移.parquet")
# 添加新列判断是专利申请权的转移还是专利权的转移
df['专利权'] = df['状态描述信息'].apply(lambda x: 1 if str(x).startswith('专利权的转移') else 0)


In [3]:
# 删除状态描述信息中"两位数字;"或"两位数字 ;"及之前的内容
import re
df['状态描述信息'] = df['状态描述信息'].apply(lambda x: re.sub(r'^.*?\d{2}\s*;', '', str(x)) if pd.notna(x) else x)


In [4]:
# 提取登记生效日并删除相关信息
import re

# 提取登记生效日
df['登记生效日'] = df['状态描述信息'].apply(
    lambda x: re.search(r'登记生效日:(\d{8})', str(x)).group(1) if re.search(r'登记生效日:(\d{8})', str(x)) else None
)

# 删除状态描述信息中的登记生效日信息
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'登记生效日:\d{8}\s*;?\s*', '', str(x)) if pd.notna(x) else x
)


In [5]:
# 统一变更前和变更后的格式
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'变更前(?!\s*;)', '变更前 ;', str(x)) if pd.notna(x) else x
)
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'变更后(?!\s*;)', '变更后 ;', str(x)) if pd.notna(x) else x
)


In [9]:
# 删除状态描述信息中的空格，并将分号替换为换行符
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: str(x).replace(' ', '').replace(';', '\n') if pd.notna(x) else x
)



In [14]:
# 确保"变更后"前面有换行符
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'(?<!\n)变更后', '\n变更后', str(x)) if pd.notna(x) else x
)
# 确保除了最开头的"变更前"外，其他"变更前"前面有换行符
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'(?<!^)(?<!\n)变更前', '\n变更前', str(x)) if pd.notna(x) else x
)


In [29]:
# 根据状态描述信息的特征设置type1字段
def set_type1(status_info):
    if pd.isna(status_info):
        return 0
    
    # 转换为字符串并按换行符分割
    lines = str(status_info).split('\n')
    
    # 检查是否以"变更前"开头且行数为6
    if len(lines) == 6 and lines[0].startswith('变更前'):
        return 1
    else:
        return 0

df['type1'] = df['状态描述信息'].apply(set_type1)

# 根据状态描述信息的特征设置type2字段
def set_type2(status_info):
    if pd.isna(status_info):
        return 0
    
    # 转换为字符串并按换行符分割
    lines = str(status_info).split('\n')
    
    # 检查是否以"变更前"开头，第二行以"专利权人"或"申请人"开头，最后一行以"地址"开头，且行数为8
    if (len(lines) == 8 and 
        lines[0] == '变更前' and 
        lines[2] == '变更前' and
        lines[4] == '变更后' and
        lines[6] == '变更后'):
        return 1
    else:
        return 0

df['type2'] = df['状态描述信息'].apply(set_type2)



In [18]:
type1 = df[df['type1'] == 1]
# 为type1数据框新增四列并提取相应信息
def extract_info(status_info):
    if pd.isna(status_info):
        return None, None, None, None
    
    lines = str(status_info).split('\n')
    
    # 初始化返回值
    前权利人 = None
    前地址 = None
    后权利人 = None
    后地址 = None
    
    # 确保有足够的行数
    if len(lines) >= 6:
        # 提取第二行":"后面的内容作为前权利人
        if ':' in lines[1]:
            前权利人 = lines[1].split(':', 1)[1].strip()
        
        # 提取第三行":"后面的内容作为前地址
        if ':' in lines[2]:
            前地址 = lines[2].split(':', 1)[1].strip()
        
        # 提取第五行":"后面的内容作为后权利人
        if ':' in lines[4]:
            后权利人 = lines[4].split(':', 1)[1].strip()
        
        # 提取第六行":"后面的内容作为后地址
        if ':' in lines[5]:
            后地址 = lines[5].split(':', 1)[1].strip()
    
    return 前权利人, 前地址, 后权利人, 后地址

# 应用函数并创建新列
type1[['前权利人', '前地址', '后权利人', '后地址']] = type1['状态描述信息'].apply(
    lambda x: pd.Series(extract_info(x))
)


C:\Users\zy222\AppData\Local\Temp\ipykernel_36084\1859307967.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  type1[['前权利人', '前地址', '后权利人', '后地址']] = type1['状态描述信息'].apply(
C:\Users\zy222\AppData\Local\Temp\ipykernel_36084\1859307967.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  type1[['前权利人', '前地址', '后权利人', '后地址']] = type1['状态描述信息'].apply(
C:\Users\zy222\AppData\Local\Temp\ipykernel_36084\1859307967.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [30]:
type2 = df[df['type2'] == 1]
# 为type2数据框新增四列并提取相应信息
def extract_info_type2(status_info):
    if pd.isna(status_info):
        return None, None, None, None
    
    lines = str(status_info).split('\n')
    
    # 初始化返回值
    前权利人 = None
    前地址 = None
    后权利人 = None
    后地址 = None
    
    # 确保有足够的行数
    if len(lines) >= 8:
        # 提取第二行":"后面的内容作为前权利人
        if ':' in lines[1]:
            前权利人 = lines[1].split(':', 1)[1].strip()
        
        # 提取第四行":"后面的内容作为前地址
        if ':' in lines[3]:
            前地址 = lines[3].split(':', 1)[1].strip()
        
        # 提取第六行":"后面的内容作为后权利人
        if ':' in lines[5]:
            后权利人 = lines[5].split(':', 1)[1].strip()
        
        # 提取第八行":"后面的内容作为后地址
        if ':' in lines[7]:
            后地址 = lines[7].split(':', 1)[1].strip()
    
    return 前权利人, 前地址, 后权利人, 后地址

# 应用函数并创建新列
type2 = type2.copy()  # 避免SettingWithCopyWarning
type2[['前权利人', '前地址', '后权利人', '后地址']] = type2['状态描述信息'].apply(
    lambda x: pd.Series(extract_info_type2(x))
)


In [20]:
type1.sample(100).to_excel("type1_sample.xlsx")
df[df['type1'] == 0].sample(100).to_excel("type0_sample.xlsx")

In [31]:
type2.sample(100).to_excel("type2_sample.xlsx")

In [33]:
df[(df['type1'] == 1) | (df['type2'] == 1)].shape[0]/df.shape[0]

0.5991292641529837

In [34]:
df[(df['type1'] == 0) & (df['type2'] == 0)].sample(100).to_excel("type0_sample.xlsx")

In [24]:
df[df['type2'] == 1].sample(100).to_excel("type2_sample.xlsx")

In [6]:
df['专利权'].value_counts(normalize=1)

专利权
1    0.756639
0    0.243361
Name: proportion, dtype: float64

In [17]:
df.sample(100).to_excel("专利申请权、专利权的转移sample.xlsx")

In [12]:
df.sample(100).to_csv("专利申请权、专利权的转移sample.csv", encoding='utf-8-sig')